In [1]:
import HelperFuncs as hfuncs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import Sequence
import h5py
import os

Using TensorFlow backend.


Define a class that allows us to sequentially load entire scans. 

In [2]:
BATCH_SIZE = 5
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17
ANGLES = 16
#Create directories for sequencer function if they don't exist
for d in ['temp/train_scan/','temp/test_scan/','temp/val_scan/']:
    if not os.path.isdir(d):
        print("Created directory: {}".format(d))
        os.makedirs(d)
        
class ScanSequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train_scan"):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
        self.angles = np.arange(0,64,64//ANGLES)
    def __len__(self):
        return self.num_batches
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        #Initialize vectors
        X_train = np.zeros((BATCH_SIZE,ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
        y_train = np.zeros((BATCH_SIZE,ZONES))
        
        j=0
        for i in range(idx*BATCH_SIZE,(idx+1)*BATCH_SIZE):
            #Download batch at index
            path = "temp/{}/batch_{}.hdf5".format(self.mode,i)
            key = "{}/batch_{}.hdf5".format(self.mode,i)
            bucket.download_file(Key=key,Filename=path)
        
            f = h5py.File(path,"r")
            try:
                X_train[j,:,:,:,:] = f['/image'].value[self.angles,:,:,:]
                y_train[j,:] = f['/labels'].value
                j += 1
            finally:
                f.close()
                os.remove(path) 
        return X_train, y_train
         



In [3]:
import h5py
from keras.models import load_model

#Load trained model
BASE_MODEL = load_model("Final_CNNRes.h5")

Look at layers 19 and 20 to see where we should begin our new model.

In [4]:
print(BASE_MODEL.layers[-19].get_config())
print(BASE_MODEL.get_layer(name='global_max_pooling2d_3').get_config())
print(BASE_MODEL.layers[-19].output_shape)
print(BASE_MODEL.layers[0].input_shape)

{'name': 'dropout_3', 'trainable': True, 'rate': 0.2}
{'name': 'global_max_pooling2d_3', 'trainable': True, 'data_format': 'channels_last'}
(None, 896)
(None, 400, 600, 1)


We know from above that our base model will output 896 dimensions. 

In [5]:
BASE_OUTPUT_DIM = BASE_MODEL.layers[-19].output_shape[1]

In [6]:
#Load validaitno data on memory so that we can visualize the weights.
UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
key_id, secret_key = hfuncs.GetAWSCredentials()
client = hfuncs.GetAWSClient(key_id,secret_key)
bucket = client.Bucket(UPLOAD_BUCKET)

#Initialize validation sequencer
mode = "val_scan"
num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

X_val = np.zeros(shape=(30,ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
y_val = np.zeros(shape=(30,ZONES))

#Val samples 1 and 2 were accidentally overwritten
for i in range(0,30):
    j = i+2
    X_val[i,:,:,:,:], y_val[i,:] = val_seq.__getitem__(j)
    




ValueError: could not broadcast input array from shape (20,8,400,600,1) into shape (8,400,600,1)

In [14]:
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM

def trainCNNResRNNAdamComplex(lr,beta1,beta2,lstm_dim,description="RNN_complex"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.get_layer('dropout_3').output)
    
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim,recurrent_dropout=0.20)(sequenced_model)

    #Finally, 17 dense layers connected to the output
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(lstm))

    out = concatenate(output_nodes)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='val_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//BATCH_SIZE,
                                           epochs=50,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =False,workers=1)
    return hist,recurrent_model

Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [15]:
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate,Activation
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras import backend as K

K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?

def trainCNNResRNNAdamSimple(lr,beta1,beta2,lstm_dim,description="RNN_simple"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.    
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim,recurrent_dropout=0)(sequenced_model)

    #Treat output of LSTM as individual guessess for each of the 17 zones
    

    out = Activation('sigmoid')(lstm)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='eval_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//5,
                                           epochs=50,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=1)
    return hist,recurrent_model

In [16]:
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate,Activation
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras import backend as K
from keras.layers.merge import Average

K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?

def trainCNNResAdamAverage(lr,beta1,beta2,lstm_dim,description="RNN_average"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.    
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)
    
    
    #One lstm layer for now
    out = Average()(sequenced_model)

    #Treat output of LSTM as individual guessess for each of the 17 zones
    

    

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='eval_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//5,
                                           epochs=100,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=1)
    return hist,recurrent_model

In [17]:
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate,Activation
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras import backend as K

K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?

def trainCNNResRNNAdamSimple(lr,beta1,beta2,lstm_dim,description="RNN_simple"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.    
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim,recurrent_dropout=0)(sequenced_model)

    #Treat output of LSTM as individual guessess for each of the 17 zones
    

    out = Activation('sigmoid')(lstm)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='eval_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//5,
                                           epochs=100,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=1)
    return hist,recurrent_model

In [18]:
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?
def trainCNNResRNNAdamComplexIsh(lr,beta1,beta2,lstm_dim,description="RNN_complexISH"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim)(sequenced_model)

    #Finally, 17 dense layers connected to the output
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(lstm))

    out = concatenate(output_nodes)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='val_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//BATCH_SIZE,
                                           epochs=50,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =False,workers=1)
    return hist,recurrent_model

Let's try training the model

In [ ]:
try:
    hist,recurrent_model = trainCNNResRNNAdamComplex(lr=0.0001,beta1=0.9,beta2=0.99,lstm_dim=800)
    del hist,recurrent_model
except:
    print("SOmething went wrong")
try:
    hist,recurrent_model = trainCNNResAdamAverage(lr=0.0001,beta1=0.9,beta2=0.99,lstm_dim=0)
    del hist,recurrent_model
except:
    print("Something went wrong")                                                  
try:
    hist,recurrent_model = trainCNNResRNNAdamSimple(lr=0.0001,beta1=0.9,beta2=0.999,lstm_dim=17)
    del hist,recurrent_model
except:
    print("Something went wrong") 
try:
    hist,recurrent_model = trainCNNResRNNAdamComplexIsh(lr=0.0001,beta1=0.9,beta2=0.999,lstm_dim=50)
    del hist,recurrent_model
except:
    print("Something went wrong") 
                                                  

Creating model...
Compiling model...
Initializing generators...
Beginning training...


In [23]:
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout

#Number of angles per scan 
ANGLES = 32

#Hidden dimensions
LSTM_OUTPUT_DIM = 1000

#Build new sequential model,Removing dense layers from model
input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
sequenced_model = TimeDistributed(BASE_MODEL.get_layer(name='global_max_pooling2d_3'))(input_scan)

#One lstm layer for now
lstm = LSTM(LSTM_OUTPUT_DIM)(sequenced_model)

#Finally, 17 dense layers connected to the output
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(lstm))

out = keras.layers.concatenate(output_nodes)

#complete model
multi_label_model = Model(input_scan, out)

